In [ ]:
import pandas as pd
dir = 'drive/MyDrive/데이터마이닝/project/Regression/data/'


## 대여소 데이터 전처리

In [ ]:
filename_rental = '공공자전거 대여소 정보(21.01.31 기준).csv'
import pandas as pd
df_station = pd.read_csv(dir+filename_rental, encoding='cp949')
df_station = df_station[['대여소 번호', '보관소(대여소)명', '위도', '경도']]
df_station

,대여소 번호,보관소(대여소)명,위도,경도
0,301,경복궁역 7번출구 앞,37.575794,126.971451
1,302,경복궁역 4번출구 뒤,37.575947,126.974060
2,303,광화문역 1번출구 앞,37.571770,126.974663
3,304,광화문역 2번출구 앞,37.572113,126.977577
4,305,종로구청 옆,37.572582,126.978355
...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),37.554569,127.145874
2150,1086,래미안 솔베뉴(구천면로),37.550598,127.145454
2151,1088,풍납사거리(강동),37.528122,127.119522
2152,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698


In [ ]:
# 대여소 정보에 TM좌표 추가
%pip install pyproj
from pyproj import Proj, transform
import numpy as np
proj_WGS84 = Proj(init='epsg:4326')
proj_UTMK = Proj(init='epsg:5178')
xs, ys = transform(proj_WGS84, proj_UTMK, np.array(df_station['경도']), np.array(df_station['위도']))
df_station['X좌표'] = xs
df_station['Y좌표'] = ys
df_station

     |████████████████████████████████| 6.6MB 5.2MB/s 


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

,대여소 번호,보관소(대여소)명,위도,경도,X좌표,Y좌표
0,301,경복궁역 7번출구 앞,37.575794,126.971451,953517.854935,1.952760e+06
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,953748.330493,1.952776e+06
2,303,광화문역 1번출구 앞,37.571770,126.974663,953798.992541,1.952312e+06
3,304,광화문역 2번출구 앞,37.572113,126.977577,954056.530364,1.952349e+06
4,305,종로구청 옆,37.572582,126.978355,954125.521044,1.952401e+06
...,...,...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,968910.925271,1.950333e+06
2150,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,968872.172914,1.949893e+06
2151,1088,풍납사거리(강동),37.528122,127.119522,966571.505860,1.947408e+06
2152,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,968986.211701,1.951002e+06


In [ ]:
# 대여소 정보에 행정동 추가
import json
import requests
def coord2dong(x, y):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(x, y)
    headers = {"Authorization": "KakaoAK 61c26d87fa0a7e09587f12e4056fd0da"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    # print(result)
    try:
        return result['documents'][1]['region_3depth_name']
    except KeyError:
        return None
   
    
dongs = []
# df_cut = df_station.iloc[0] 
# longtitude = df_cut['경도']
# latitude = df_cut['위도']
# print(longtitude, latitude)
# coord2dong(float(longtitude), float(latitude))

count=0
for station in df_station.iloc:
    print(count)
    dongs.append(coord2dong(float(station['경도']), float(station['위도'])))
    count+=1
    
df_station['행정동'] = dongs
df_station


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,대여소 번호,보관소(대여소)명,위도,경도,X좌표,Y좌표,행정동
0,301,경복궁역 7번출구 앞,37.575794,126.971451,953517.854935,1.952760e+06,사직동
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,953748.330493,1.952776e+06,사직동
2,303,광화문역 1번출구 앞,37.571770,126.974663,953798.992541,1.952312e+06,사직동
3,304,광화문역 2번출구 앞,37.572113,126.977577,954056.530364,1.952349e+06,종로1.2.3.4가동
4,305,종로구청 옆,37.572582,126.978355,954125.521044,1.952401e+06,종로1.2.3.4가동
...,...,...,...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,968910.925271,1.950333e+06,명일1동
2150,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,968872.172914,1.949893e+06,명일1동
2151,1088,풍납사거리(강동),37.528122,127.119522,966571.505860,1.947408e+06,성내1동
2152,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,968986.211701,1.951002e+06,고덕1동


In [ ]:
df_station

,대여소 번호,보관소(대여소)명,위도,경도,X좌표,Y좌표,행정동
0,301,경복궁역 7번출구 앞,37.575794,126.971451,953517.854935,1.952760e+06,사직동
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,953748.330493,1.952776e+06,사직동
2,303,광화문역 1번출구 앞,37.571770,126.974663,953798.992541,1.952312e+06,사직동
3,304,광화문역 2번출구 앞,37.572113,126.977577,954056.530364,1.952349e+06,종로1.2.3.4가동
4,305,종로구청 옆,37.572582,126.978355,954125.521044,1.952401e+06,종로1.2.3.4가동
...,...,...,...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,968910.925271,1.950333e+06,명일1동
2150,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,968872.172914,1.949893e+06,명일1동
2151,1088,풍납사거리(강동),37.528122,127.119522,966571.505860,1.947408e+06,성내1동
2152,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,968986.211701,1.951002e+06,고덕1동


In [ ]:
# df_station['행정동'].to_csv('drive/MyDrive/데이터마이닝/project/Regression/data/대여소_행정동동.csv', mode='w', encoding='euc-kr')

In [ ]:
# # 대여소 행정동 csv 불러오기
# station_hgd = pd.read_csv('drive/MyDrive/데이터마이닝/project/Regression/data/대여소_행정동동.csv', encoding='euc-kr', index_col=0)
# station_hgd

,행정동
0,내자동
1,적선동
2,당주동
3,세종로
4,세종로
...,...
2149,명일동
2150,명일동
2151,성내동
2152,고덕동


In [ ]:
# # 대여소 행정동 concat
# station_concat = pd.concat([df_station, station_hgd], axis=1)
# # station_concat['행정동_결측'] = station_concat['행정동'].isnull()
# # station_concat[station_concat['행정동_결측']==True]
# station_concat

In [ ]:
# # 대여소 행정동 변환
# station_concat = station_concat.dropna()
# station_hgd_chg = pd.read_csv('drive/MyDrive/데이터마이닝/project/Regression/data/대여소_행정동_변환.csv', encoding='euc-kr', index_col=1)
# station_hgd_chg = station_hgd_chg['변환후'].to_dict()
# station_hgd_chg
# # 대여소 정보에 행정동 변환 추가

# new_dongs = []
# count = 0
# for station in station_concat.iloc:
#     # print(station)
#     print(count)
#     new_dongs.append(station_hgd_chg[station['행정동']])
#     count+=1
# station_concat['행정동_변환'] = new_dongs
# station_concat

##인구조사 데이터 전처리

In [ ]:
#인구조사 csv 불러오기
filename = '인구조사.csv'
df_people = pd.read_csv(dir+filename, encoding='euc-kr', index_col = 0)

df_people = df_people[df_people['구분']=='계']
df_people = df_people[~ df_people['동'].isin(['합계','소계'])]
df_people
# df_people.to_csv('drive/MyDrive/데이터마이닝/project/Regression/data/인구조사.csv', mode='w', encoding='euc-kr')

,기간,자치구,동,구분,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상+
6,2021.1/4,종로구,사직동,계,9813,221,357,352,350,506,759,788,743,763,811,838,756,735,507,397,377,303,166,57,20,7
9,2021.1/4,종로구,삼청동,계,2803,46,93,112,107,142,178,169,213,227,201,205,222,247,190,157,115,89,56,23,5,6
12,2021.1/4,종로구,부암동,계,10069,197,333,426,482,652,748,652,671,755,894,899,807,790,538,430,346,255,121,58,11,4
15,2021.1/4,종로구,평창동,계,18491,473,672,765,862,1151,1289,1048,1226,1288,1484,1674,1633,1519,1088,759,614,498,278,118,42,10
18,2021.1/4,종로구,무악동,계,8425,275,429,492,506,485,439,418,542,662,810,773,640,574,429,334,273,205,93,35,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,2021.1/4,강동구,둔촌1동,계,146,0,3,4,3,8,13,4,10,18,9,12,9,20,13,6,4,4,2,2,1,1
1341,2021.1/4,강동구,둔촌2동,계,25309,743,980,927,1171,1574,1969,1829,1899,1864,1885,2139,2100,2170,1589,1069,658,393,221,89,28,12
1344,2021.1/4,강동구,암사1동,계,35245,965,1195,1172,1404,2128,2729,2541,2732,2765,2810,2974,2997,2920,2136,1636,1091,601,305,117,19,8
1347,2021.1/4,강동구,천호2동,계,34526,894,823,844,995,1795,3153,3255,3050,2490,2525,2778,3019,2983,2176,1581,1113,661,255,102,32,2


In [ ]:
# #인구조사 행정동 변환
# df_people_cut = df_people[df_people['구분']=='계']
# df_people_chg = df_people_cut[~ df_people_cut['동'].isin(['합계','소계'])]
# df_people_chg
# people_hgd_chg = pd.read_csv('drive/MyDrive/데이터마이닝/project/Regression/data/인구조사_행정동_변환.csv', encoding='euc-kr', index_col=1)
# people_hgd_chg = people_hgd_chg['변환후'].to_dict()
# people_hgd_chg
# # # 대여소 정보에 행정동 변환 추가

# new_dongs = []
# count = 0
# for people_dong in df_people_chg.iloc:
#     # print(station)
#     # print(count)
#     new_dongs.append(people_hgd_chg[people_dong['동']])
#     count+=1
#     break
# # df_people_chg['행정동_변환'] = new_dongs
# # df_people_chg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,기간,자치구,동,구분,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상+,행정동_변환
6,2021.1/4,종로구,사직동,계,9813,221,357,352,350,506,759,788,743,763,811,838,756,735,507,397,377,303,166,57,20,7,사직동
9,2021.1/4,종로구,삼청동,계,2803,46,93,112,107,142,178,169,213,227,201,205,222,247,190,157,115,89,56,23,5,6,삼청동
12,2021.1/4,종로구,부암동,계,10069,197,333,426,482,652,748,652,671,755,894,899,807,790,538,430,346,255,121,58,11,4,부암동
15,2021.1/4,종로구,평창동,계,18491,473,672,765,862,1151,1289,1048,1226,1288,1484,1674,1633,1519,1088,759,614,498,278,118,42,10,평창동
18,2021.1/4,종로구,무악동,계,8425,275,429,492,506,485,439,418,542,662,810,773,640,574,429,334,273,205,93,35,8,3,무악동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,2021.1/4,강동구,둔촌1동,계,146,0,3,4,3,8,13,4,10,18,9,12,9,20,13,6,4,4,2,2,1,1,둔촌동
1341,2021.1/4,강동구,둔촌2동,계,25309,743,980,927,1171,1574,1969,1829,1899,1864,1885,2139,2100,2170,1589,1069,658,393,221,89,28,12,둔촌동
1344,2021.1/4,강동구,암사1동,계,35245,965,1195,1172,1404,2128,2729,2541,2732,2765,2810,2974,2997,2920,2136,1636,1091,601,305,117,19,8,암사동
1347,2021.1/4,강동구,천호2동,계,34526,894,823,844,995,1795,3153,3255,3050,2490,2525,2778,3019,2983,2176,1581,1113,661,255,102,32,2,천호동


In [ ]:
# #행정동별 인구수 합계
# df_people_group = df_people_chg.drop(columns = ['자치구','기간','동', '구분'])
# df_people_group = df_people_group.groupby('행정동_변환')
# df_people_group = df_people_group.agg('sum')
# df_people_group


,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상+
행정동_변환,,,,,,,,,,,,,,,,,,,,,,
가락동,87502,3273,4033,3777,3833,5317,6789,7125,7582,7006,6934,7065,7017,6731,4558,2973,1669,989,569,200,50,12
가리봉동,13447,210,237,185,191,571,1158,1030,933,840,1148,1518,1975,1329,816,536,348,236,131,38,14,3
가산동,25863,400,374,329,443,1964,4950,3675,2354,1590,1585,1829,2082,1660,994,748,445,261,123,43,11,3
가양동,64201,1503,1364,1234,1693,3767,8117,8101,5898,4189,3869,4000,4632,5328,3929,2550,1744,1256,678,269,69,11
가회동,4297,68,126,174,159,225,271,258,322,348,413,385,333,339,250,220,179,123,63,32,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
회현동,5738,78,154,115,124,235,336,308,329,353,439,498,583,622,502,338,286,193,111,66,23,45
효창동,10369,402,378,344,402,697,930,949,964,756,822,878,738,632,435,358,291,224,111,48,7,3
후암동,17843,410,560,594,682,1030,1602,1437,1453,1338,1509,1485,1401,1323,982,753,563,403,195,72,34,17


In [ ]:
# df_people_group

In [ ]:
# df_people_group.to_csv('drive/MyDrive/데이터마이닝/project/Regression/data/인구조사_행정동_집계.csv', mode='w', encoding='euc-kr')

## 대여소 자료 + 인구조사 자료

In [ ]:
df_station

,대여소 번호,보관소(대여소)명,위도,경도,X좌표,Y좌표,행정동
0,301,경복궁역 7번출구 앞,37.575794,126.971451,953517.854935,1.952760e+06,사직동
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,953748.330493,1.952776e+06,사직동
2,303,광화문역 1번출구 앞,37.571770,126.974663,953798.992541,1.952312e+06,사직동
3,304,광화문역 2번출구 앞,37.572113,126.977577,954056.530364,1.952349e+06,종로1.2.3.4가동
4,305,종로구청 옆,37.572582,126.978355,954125.521044,1.952401e+06,종로1.2.3.4가동
...,...,...,...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,968910.925271,1.950333e+06,명일1동
2150,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,968872.172914,1.949893e+06,명일1동
2151,1088,풍납사거리(강동),37.528122,127.119522,966571.505860,1.947408e+06,성내1동
2152,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,968986.211701,1.951002e+06,고덕1동


In [ ]:
df_people[df_people['자치구']=='서대문구']

,기간,자치구,동,구분,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상+
618,2021.1/4,서대문구,천연동,계,17596,594,628,694,759,1044,1199,1091,1308,1267,1548,1578,1455,1261,945,725,656,503,247,73,16,5
621,2021.1/4,서대문구,홍제1동,계,24257,686,839,836,1040,1496,1693,1658,1778,1688,2059,2121,1965,1902,1429,1118,884,662,281,104,15,3
624,2021.1/4,서대문구,홍제3동,계,15725,344,412,448,651,1039,1088,1028,958,927,1244,1411,1261,1297,1044,859,756,577,274,68,17,22
627,2021.1/4,서대문구,홍제2동,계,14104,357,425,496,578,853,997,982,918,899,1136,1254,1297,1228,813,674,553,388,175,63,13,5
630,2021.1/4,서대문구,홍은1동,계,23111,658,831,768,1015,1437,1508,1423,1640,1593,1858,2030,1948,1921,1423,1076,869,653,320,111,23,6
633,2021.1/4,서대문구,홍은2동,계,27523,538,722,937,1259,2177,2329,1809,1778,1736,2260,2435,2226,2201,1605,1219,1008,730,385,108,34,27
636,2021.1/4,서대문구,남가좌1동,계,15861,780,1353,867,602,731,809,907,1551,1684,1407,1178,1005,892,687,526,405,286,129,49,9,4
639,2021.1/4,서대문구,남가좌2동,계,30658,1161,1503,1144,1282,2520,2371,2035,2573,2447,2488,2420,2037,1920,1483,1211,975,658,290,104,34,2
642,2021.1/4,서대문구,북가좌1동,계,18455,610,801,1038,920,1037,1138,1119,1289,1550,1719,1620,1419,1274,945,765,551,404,174,66,14,2
645,2021.1/4,서대문구,북가좌2동,계,32647,660,986,1030,1383,2065,2792,2113,2186,2344,2711,2760,2657,2681,2014,1659,1277,789,378,125,33,4


In [ ]:
 merge_stat_peo = pd.merge(df_station,df_people, how='left',left_on='행정동', right_on='동')
 merge_stat_peo

,대여소 번호,보관소(대여소)명,위도,경도,X좌표,Y좌표,행정동,기간,자치구,동,구분,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상+
0,301,경복궁역 7번출구 앞,37.575794,126.971451,953517.854935,1.952760e+06,사직동,2021.1/4,종로구,사직동,계,9813.0,221.0,357.0,352.0,350.0,506.0,759.0,788.0,743.0,763.0,811.0,838.0,756.0,735.0,507.0,397.0,377.0,303.0,166.0,57.0,20.0,7.0
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,953748.330493,1.952776e+06,사직동,2021.1/4,종로구,사직동,계,9813.0,221.0,357.0,352.0,350.0,506.0,759.0,788.0,743.0,763.0,811.0,838.0,756.0,735.0,507.0,397.0,377.0,303.0,166.0,57.0,20.0,7.0
2,303,광화문역 1번출구 앞,37.571770,126.974663,953798.992541,1.952312e+06,사직동,2021.1/4,종로구,사직동,계,9813.0,221.0,357.0,352.0,350.0,506.0,759.0,788.0,743.0,763.0,811.0,838.0,756.0,735.0,507.0,397.0,377.0,303.0,166.0,57.0,20.0,7.0
3,304,광화문역 2번출구 앞,37.572113,126.977577,954056.530364,1.952349e+06,종로1.2.3.4가동,2021.1/4,종로구,종로1.2.3.4가동,계,8449.0,114.0,145.0,138.0,172.0,477.0,678.0,603.0,572.0,592.0,672.0,683.0,828.0,833.0,646.0,454.0,399.0,240.0,140.0,38.0,20.0,5.0
4,305,종로구청 옆,37.572582,126.978355,954125.521044,1.952401e+06,종로1.2.3.4가동,2021.1/4,종로구,종로1.2.3.4가동,계,8449.0,114.0,145.0,138.0,172.0,477.0,678.0,603.0,572.0,592.0,672.0,683.0,828.0,833.0,646.0,454.0,399.0,240.0,140.0,38.0,20.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,968910.925271,1.950333e+06,명일1동,2021.1/4,강동구,명일1동,계,26123.0,959.0,1161.0,1147.0,1327.0,1579.0,1643.0,1861.0,2259.0,1999.0,2226.0,2125.0,1926.0,2039.0,1583.0,1035.0,578.0,362.0,201.0,92.0,17.0,4.0
2155,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,968872.172914,1.949893e+06,명일1동,2021.1/4,강동구,명일1동,계,26123.0,959.0,1161.0,1147.0,1327.0,1579.0,1643.0,1861.0,2259.0,1999.0,2226.0,2125.0,1926.0,2039.0,1583.0,1035.0,578.0,362.0,201.0,92.0,17.0,4.0
2156,1088,풍납사거리(강동),37.528122,127.119522,966571.505860,1.947408e+06,성내1동,2021.1/4,강동구,성내1동,계,20309.0,713.0,865.0,799.0,934.0,1172.0,1607.0,1560.0,1778.0,1679.0,1677.0,1589.0,1461.0,1487.0,1130.0,846.0,491.0,299.0,149.0,54.0,17.0,2.0
2157,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,968986.211701,1.951002e+06,고덕1동,2021.1/4,강동구,고덕1동,계,25500.0,976.0,1955.0,2096.0,1377.0,1195.0,1213.0,1268.0,2134.0,2868.0,2542.0,1798.0,1642.0,1616.0,1162.0,695.0,395.0,267.0,158.0,97.0,17.0,29.0


In [ ]:
merge_stat_peo_drop = merge_stat_peo.dropna()

In [ ]:
merge_stat_peo_drop.to_csv('drive/MyDrive/데이터마이닝/project/Regression/data/대여소별 주변 인구.csv', mode='w', encoding='euc-kr')